# 전체(사전 훈련 없이)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import copy
from PIL import Image
import os
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import random

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

from tqdm import tqdm

In [2]:
v = '3'
epochs = 36

In [3]:
# 완벽한 실험 재현성을 위한 랜덤제어
random_seed = 28
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU

In [4]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3060


In [5]:
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(224),
        transforms.Resize(256),
        transforms.CenterCrop(224),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),    
}

In [6]:
image_path = "C:/Users/ANDlab3/Desktop/final train/data/vision_data/"
image_datasets = {x: datasets.ImageFolder(os.path.join(image_path, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val','test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

In [7]:
dataset_sizes

{'train': 6475, 'val': 925, 'test': 1850}

In [8]:
class_num = len(class_names)
class_num

37

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
pre_model = models.resnet50(pretrained=False)
num_ftrs = pre_model.fc.in_features

#Changing the number of outputs in the last layer to the number of different item types
pre_model.fc = nn.Linear(num_ftrs, 1000)

In [11]:
class vision_module(nn.Module):
    def __init__(self, pre_model):
        super(vision_module, self).__init__()
    
        D_in, H, D_out = 1000, 500, 37
        self.resnet50 = pre_model
        
        self.dense = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(D_in, H)
        )
        
        self.classifier = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(H, D_out)
        )
    
    def forward(self, image):
        outputs = self.resnet50(image)
        
        dense = self.dense(outputs)
        
        fc = self.classifier(dense)
        
        return  fc

In [12]:
model_ft = vision_module(pre_model)
model_ft.to(device)

vision_module(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
   

In [13]:
# # 파라메타 번호 확인 하기
# i = 0
# for name, param in model_ft.named_parameters():
    
#     print(i,name)
#     i+= 1

In [14]:
# for i, (name, param) in enumerate(model_ft.named_parameters()):
    
#     param.requires_grad = False
#     if i == 158:
#         print('end')
#         break

In [15]:
# for p in model_ft.named_parameters():
#     print(p)
# #     break

In [16]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [17]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epochs)

Epoch 0/35
----------
train Loss: 3.2733 Acc: 0.0653
val Loss: 3.2612 Acc: 0.0649

Epoch 1/35
----------
train Loss: 2.5975 Acc: 0.1914
val Loss: 2.1550 Acc: 0.2519

Epoch 2/35
----------
train Loss: 2.1365 Acc: 0.2922
val Loss: 1.7075 Acc: 0.3978

Epoch 3/35
----------
train Loss: 1.8065 Acc: 0.3787
val Loss: 1.5164 Acc: 0.4357

Epoch 4/35
----------
train Loss: 1.6308 Acc: 0.4346
val Loss: 1.3393 Acc: 0.5200

Epoch 5/35
----------
train Loss: 1.4863 Acc: 0.4726
val Loss: 1.4334 Acc: 0.4951

Epoch 6/35
----------
train Loss: 1.4547 Acc: 0.4975
val Loss: 1.4228 Acc: 0.5438

Epoch 7/35
----------
train Loss: 1.2119 Acc: 0.5668
val Loss: 1.0228 Acc: 0.6270

Epoch 8/35
----------
train Loss: 1.1310 Acc: 0.5924
val Loss: 0.9945 Acc: 0.6368

Epoch 9/35
----------
train Loss: 1.0836 Acc: 0.6031
val Loss: 0.9637 Acc: 0.6443

Epoch 10/35
----------
train Loss: 1.0713 Acc: 0.6060
val Loss: 1.0170 Acc: 0.6422

Epoch 11/35
----------
train Loss: 1.0454 Acc: 0.6176
val Loss: 0.9932 Acc: 0.6605

Ep

In [18]:
SAVE_PATH = "C:/Users/ANDlab3/Desktop/final train/model/vision_model/"
torch.save(model_ft.state_dict(), SAVE_PATH + 'vision_model_fine_tuned'+v+'.pt')

# Test

In [19]:
correct = 0
total = 0

true = []
pred = []

with torch.no_grad():
    model_ft.eval() #현재는 모델에 드랍아웃이나, 패딩이 없어서 필요없지만 만약 사용된 경우에는 eval을 통해서 평가시에는 꼭 비활성화 시켜야한다.
    f1_score = 0
    for data in dataloaders['test']:
        images, labels = data[0].to(device), data[1].to(device)
        
        outputs =  model_ft(images)
        _, predicted = torch.max(outputs.data, 1)
        
        
        total += labels.size(0) # 개수 누적(총 개수)
        correct += (predicted == labels).sum().item() # 누적(맞으면 1, 틀리면 0으로 합산)
  
        true += labels.cpu().numpy().tolist()
        pred += predicted.cpu().numpy().tolist()

print('done')

done


In [20]:
accuracy = accuracy_score(true, pred)
round((accuracy * 100), 4)

67.9459

In [21]:
print('전체 데이터 수 : ', total)
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

전체 데이터 수 :  1850
Accuracy of the network on the 10000 test images: 67 %


In [22]:
# Classification Report 저장
REPORT_PATH = "C:/Users/ANDlab3/Desktop/final train/report/vision_report/"
CL_REPORT_FILE = REPORT_PATH + "cl_report_"+v+".csv"

cl_report = classification_report(true, pred, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(4)
cl_report_df.to_csv(CL_REPORT_FILE)
print(cl_report_df)

              precision  recall  f1-score    support
0                0.7302  0.9200    0.8142    50.0000
1                0.9615  1.0000    0.9804    50.0000
2                0.8000  0.6400    0.7111    50.0000
3                0.8235  0.8400    0.8317    50.0000
4                0.7636  0.8400    0.8000    50.0000
5                0.3333  0.1200    0.1765    50.0000
6                0.7885  0.8200    0.8039    50.0000
7                0.7963  0.8600    0.8269    50.0000
8                0.2308  0.0600    0.0952    50.0000
9                0.9592  0.9400    0.9495    50.0000
10               0.5333  0.6400    0.5818    50.0000
11               0.7692  0.8000    0.7843    50.0000
12               0.5488  0.9000    0.6818    50.0000
13               0.8837  0.7600    0.8172    50.0000
14               0.6410  0.5000    0.5618    50.0000
15               0.3200  0.1600    0.2133    50.0000
16               0.7241  0.4200    0.5316    50.0000
17               0.5062  0.8200    0.6260    5